# Data Journalism Lesson 4: Aggregates, Part 2

Learn how to take lots of little things and make numbers out of bigger things.

In [ ]:
# Setup code for the notebook
import pandas as pd
from IPython.display import display, HTML

# Helper functions for grading exercises
def check_head(result, expected_shape=(5, 103)):
    """Check if the head() method was used correctly."""
    # Check if the result is a DataFrame and has the expected shape (default 5 rows)
    if isinstance(result, pd.DataFrame) and result.shape[0] == expected_shape[0]:
        display(HTML('<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;">' +
                 '<strong>Great work!</strong> You displayed the first few rows.</div>'))
    else:
        display(HTML('<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;">' +
                 '<strong>Not quite!</strong> Make sure you are calling .head() on your DataFrame.</div>'))

def check_groupby_sum(result, expected):
    """Check if the result of a groupby/sum operation matches the expected result."""
    try:
        pd.testing.assert_frame_equal(result.reset_index(), expected.reset_index(), check_dtype=False, check_like=True, rtol=1e-3)
        display(HTML('<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;">' +
                 '<strong>Great work!</strong> You correctly grouped, counted, summed, and sorted.</div>'))
    except AssertionError as e:
        # print(e) # Uncomment for debugging
        display(HTML('<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;">' +
                 '<strong>Not quite!</strong> Check your grouping column, aggregation functions (count, sum), and sorting column/order.</div>'))

def check_groupby_mean(result, expected):
    """Check if the result of a groupby/mean operation matches the expected result."""
    try:
        # Ensure columns match before comparison, ignore index for this check
        pd.testing.assert_frame_equal(result.reset_index(), expected.reset_index(), check_dtype=False, check_like=True, rtol=1e-3)
        display(HTML('<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;">' +
                 '<strong>Great work!</strong> You correctly grouped, aggregated (count, sum, mean), and sorted.</div>'))
    except AssertionError as e:
        # print(e) # Uncomment for debugging
        display(HTML('<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;">' +
                 '<strong>Not quite!</strong> Check your grouping column, aggregation functions (count, sum, mean), and sorting column/order.</div>'))

In [ ]:
# Load required data and prepare expected results for checks
import pandas as pd

state = "Minnesota" # Using Minnesota like the previous notebooks
state_lower = state.lower().replace(' ', '-')

# Construct the path relative to the notebook location
file_path = f'../_static/nursing-homes/{state_lower}.csv'
homes = pd.read_csv(file_path)

# Calculate expected results for checks
expected_sum = (homes
    .groupby('county_parish')
    .agg(
        total_homes=('cms_certification_number_ccn', 'count'),
        total_beds=('number_of_certified_beds', 'sum')
    )
    .sort_values('total_beds', ascending=False)
)

expected_mean = (homes
    .groupby('county_parish')
    .agg(
        total_homes=('cms_certification_number_ccn', 'count'),
        total_beds=('number_of_certified_beds', 'sum'),
        average_rating=('overall_rating', 'mean')
    )
    .sort_values('average_rating', ascending=False)
)

# Variables for glue
most_beds_county = expected_sum.index[0]
most_beds_count = expected_sum.iloc[0]['total_beds']
most_beds_homes = expected_sum.iloc[0]['total_homes']

highest_avg_rating_county = expected_mean.index[0]
highest_avg_rating_value = expected_mean.iloc[0]['average_rating']
highest_avg_rating_homes = expected_mean.iloc[0]['total_homes']

county_language_singular = "county" # Default, adjust if needed for other states
county_language_plural = "counties"

In [ ]:
from myst_nb import glue

# Glue variables for use elsewhere in the notebook
glue("state", state, display=False)
glue("most_beds_county", most_beds_county, display=False)
glue("most_beds_count", f"{int(most_beds_count):,}", display=False)
glue("most_beds_homes", f"{int(most_beds_homes):,}", display=False)
glue("highest_avg_rating_county", highest_avg_rating_county, display=False)
glue("highest_avg_rating_value", f"{highest_avg_rating_value:.2f}", display=False)
glue("highest_avg_rating_homes", f"{int(highest_avg_rating_homes):,}", display=False)
glue("county_language_singular", county_language_singular, display=False)
glue("county_language_plural", county_language_plural, display=False)

## The Goal

In this lesson, you will learn about aggregates ... again. Just like the last tutorial, you'll understand how to group data together and perform calculations on those groups using Python and the pandas library. But this time, instead of just counting, we're going to create different, but just as simple, numbers from our data. Things like sums, averages or medians. These skills are essential for summarizing large datasets and uncovering patterns in your data.

## What is Data Journalism?

Ben Welsh, the news applications editor at Reuters, has been in the game a long time. He learned data journalism as a student, as a reporter and later a graphics and data editor. He was part of the movement of data journalism to expand from reporters who know data to reporters who know code and build websites.

The one thing data journalists across the country know him for is his First Notebook class, which is packed every year at the NICAR conference. There's a waiting list to get in most years. People travel, often on their own dime, to sit in a conference room of a hotel in a city somewhere for eight hours to learn how to write code that does basic data analysis. Why? Because he's good at it, that's why.

For Welsh, a news application or a data-driven investigation have a logic to them.

"I often think about (them) as kind of pipelines, that are in a sequential order trying to take, or in assembly lines, that are trying to take the raw materials of data that we're gathering from the world or wherever else," he said. "We're refining it, cleaning it, preparing it, simplifying it and summarizing it so that we can create the manufactured good. A refined piece of information that's ready for people to buy, to consume, and it's something that they want, right?

"Those information factors, those pipelines that do that data delivery, are nothing but a collection of these very basic fundamental tools that you're trying to teach, which is how do I get data in and read it out? How do I turn it around and change its shape and summarize and aggregate it into something that's separate?"

Sensing a pattern yet? You should. They're everywhere in these tutorials. But one that should become very clear in what follows is that simple things like grouping things together and calculating up a number is very easy and very powerful.

## The Basics

Hopefully, starting today, lots of what you are doing will start to feel like a pattern. We're going to start the exact same way as last time, and every time after this. First, we need to load the libraries we need. In Python for data analysis, that almost always means pandas.

We already imported pandas as `pd` in the setup cell at the top of this notebook. **If `import pandas as pd` isn't near the top of your notebook, you're probably doing it wrong.**

And, if your next bit of code isn't loading in your data, you're most likely doing it wrong here as well. A simple analogy -- we want to build a house. What do you need? First, tools. Second, building materials. Libraries are tools, data are the building materials.

Just like the previous tutorial, we've loaded the nursing home data for {glue:text}`state` into a DataFrame called `homes`.

### Exercise 1: Using head to see your data ... again

We're going to need to see our data again, so we refresh our memory of what it looks like and what the column names are. This should start to feel a little familiar.

**Call the `.head()` method on the DataFrame we created above (`homes`)** to show the headers and **the first few rows of data**.

In [ ]:
# Use the .head() method on the 'homes' DataFrame
head_result = ____

# Display the result
display(head_result)

# Check your answer
check_head(head_result)

Things to note here -- we're going to look at some numbers for each county. So you'll need the column name that has the county or parish name in it (`county_parish`), the number of beds (`number_of_certified_beds`), and the overall rating (`overall_rating`). Keep those handy.

### Exercise 2: Group by and count ... and more

In the last tutorial, we grouped and counted. We should do that again -- it's useful to know how many homes are in a county when you're calculating other numbers. But this time, we're going to not just count the homes, we're going to total up the number of beds in a county using `sum()`. This sounds like a silly number if you've never had to think about it, and until the pandemic, most people never thought about how many nursing home beds there were in a county. But one group of people who were particularly hit hard by the early days of the pandemic were nursing home residents.

And the critical idea here is the **difference between a count and a sum**. You learned how to count before you went to school -- 1, 2, 3, 4. But if I said what is the sum of those numbers, you wouldn't say 4. 1+2+3+4 = 10. It seems silly, but you'd be surprised how many people get too far into worrying about code and forget this distinction.

:::{admonition} Common Mistake
:class: caution
A sum is not a count. And an editor will never, ever say to you "make a sum of" something. They'll use words like "total up" or ask a question starting with "how many". Pay attention to how other people talk about the action of creating numbers -- a key skill is connecting their words to the code you need to write.
:::

Good news: making a sum is very easy. In the last tutorial, you made a count of homes. We'll use the `.agg()` method after `.groupby()` to perform multiple aggregations at once. `.agg()` takes a dictionary where keys are the new column names and values specify the column to aggregate and the function to use (like 'count', 'sum', 'mean', etc.).

Let's keep our count from before, but this time we're going to add a sum of beds. We'll use `.agg()` to calculate both `total_homes` (a count) and `total_beds` (a sum). Then, we'll arrange it by which county has the most beds using `.sort_values()`.

**You fill in where there are blanks with what needs to be there.** What you fill in are the hints from above.

In [ ]:
# Group by county, aggregate count and sum, then sort
homes_grouped_sum = (____  # Start with the DataFrame
    .groupby(____) # Group by the county column
    .agg(
        total_homes=(____, 'count'), # Count homes using a unique ID column
        total_beds=(____, 'sum')     # Sum the number of beds column
    )
    .sort_values(by=____, ascending=False) # Sort by total beds, descending
)

# Display the result
display(homes_grouped_sum.head())

# Check your answer
check_groupby_sum(homes_grouped_sum, expected_sum)

In {glue:text}`most_beds_county` {glue:text}`county_language_singular`, {glue:text}`state`, there's {glue:text}`most_beds_homes` nursing homes with {glue:text}`most_beds_count` beds available. When the pandemic hit, this alone would have been a big part of a story about how nursing homes are vulnerable and people are dying. I, for one, am glad those days are behind us.

But what have we done again? Is the same county with the most homes also the one with the most beds? Have we made a population map again? We have. Time to start moving away from that.

### Exercise 3: A mean is an average

Two things to notice about what we just did: First, when we needed another number, we just added another entry to our `.agg()` dictionary. It's really no more difficult than that. Second: note the pattern. `new_column_name=('column_to_aggregate', 'aggregation_function')`. Give your number a name, specify the original column, and pick a math function (as a string) to create that number.

To prove it, let's find out which {glue:text}`county_language_singular` has the best-rated homes by overall rating in {glue:text}`state`.

To do that, we're just going to add another line to our `.agg()` call. The important thing to learn this time is that pandas often uses statistical terms for function names. You will never, ever hear a normal person call an average a mean, but in mathematics and statistics, that's what an average often is. A mean. Any time there is a math word for something that normal people don't use, there's a very, very good chance that the function is called that math word, not the normal people word.

You will never, ever hear an editor say "What about the mean rating?" But they will ask you about the average. The sooner you connect the dots on those two things, the better off you'll be. The function name in pandas for average is `mean`.

Remember: We're trying to find the *average* of the `overall_rating`. Let's arrange it by the average rating when we're done creating that average.

In [ ]:
# Group by county, aggregate count, sum, and mean, then sort
homes_grouped_mean = (____  # Start with the DataFrame
    .groupby(____) # Group by the county column
    .agg(
        total_homes=(____, 'count'), # Count homes
        total_beds=(____, 'sum'),     # Sum beds
        average_rating=(____, ____)  # Calculate the mean of the overall rating
    )
    .sort_values(by=____, ascending=False) # Sort by average rating, descending
)

# Display the result
display(homes_grouped_mean.head())

# Check your answer
check_groupby_mean(homes_grouped_mean, expected_mean)

The {glue:text}`county_language_singular` with the highest rated nursing homes in {glue:text}`state` is {glue:text}`highest_avg_rating_county` with an average score of {glue:text}`highest_avg_rating_value`. One weakness of this approach: What happens if you have one home in a {glue:text}`county_language_singular` with a 5-star rating? That {glue:text}`county_language_singular` will obviously have the highest average. This is why we left the count (`total_homes`) in there. It offers context for us to evaluate our findings. We'll learn later how to deal with this by filtering out groups below a certain threshold.

Now, we could keep doing this with more numbers -- and later, other numbers will become useful. For instance, we could calculate a median with `'median'`. What's the difference between a mean and a median? Both measure the middle of a group of numbers. You've calculated averages since you were a kid -- add up all the numbers, then divide it by the number of numbers. The problem with averages? They can be distorted by extremes. Let's pretend there's 20 students in a class, and every single student in the class has a net worth of \$15,000. So the average is, obviously, \$15,000. If we added in Elon Musk to the class, who as of this writing had a net worth of \$241 billion, the average net worth in the class is now \$11.5 billion. Do you think any student in that class feels like they are average? Not hardly, but they are very, very, very happy if that is now their net worth.

Enter, the median. The median is where half the numbers are above, half are below. It does not matter how high the top number is or how low the bottom number is. The median is the number right in the middle. Or, in our fictional class, it's still $15,000. The number wasn't influenced by the gargantuan extreme that Elon Musk represents.

## The Recap

In data analysis with pandas, `groupby()` and `agg()` (or related methods like `count()`, `sum()`, `mean()`) are two of the most basic, but most common tools. With these functions, you can take national data and make it local. You can calculate new numbers that shine new light on an issue, in ways readers can easily understand. In just a few lessons, you've learned a *huge* amount of basic data analysis. You'll use this pattern *a lot*. What remains is creative application of what you've learned.

## Terms to Know

- **pandas**: The primary Python library for data manipulation and analysis.
- **DataFrame**: The main data structure in pandas, similar to a table or spreadsheet.
- **groupby()**: A pandas method used to group rows in a DataFrame based on the values in one or more columns.
- **agg()**: A pandas method used after `groupby()` to apply one or more aggregation functions (like count, sum, mean, median) to specified columns of each group.
- **mean()**: An aggregation function (used as a string `'mean'` in `agg()`) that calculates the arithmetic average of a set of numbers.
- **median()**: An aggregation function (used as a string `'median'` in `agg()`) that finds the middle value in a sorted set of numbers.
- **sum()**: An aggregation function (used as a string `'sum'` in `agg()`) that adds up all the values in a specified column.
- **count()**: An aggregation function (used as a string `'count'` in `agg()`) that counts the number of non-missing values in a specified column for each group.
- **size()**: A pandas method used after `groupby()` that counts the total number of rows in each group (including missing values).
- **sort_values()**: A pandas method used to order rows in a DataFrame based on values in specified columns.
- **ascending=False**: An argument used within `sort_values()` to specify descending order for sorting.